## Imports

In [1]:
########################################################################
## FOR NOTEBOOKS ONLY: ADD THE PROJECT ROOT TO THE PYTHON PATH
########################################################################

import os
import sys

sys.path.insert(
    0, os.path.abspath(os.path.join(os.getcwd(), '..'))
)

In [2]:
import datetime

from sinkingfund.allocation.sorted import SortedAllocator
from sinkingfund.utils.loaders import load_envelopes_from_csv, load_bills_from_csv
from sinkingfund.schedules.indep_scheduler import IndependentScheduler
from sinkingfund.schedules.lp_scheduler import LPScheduler

Set exogenous details.

In [3]:
# Set bill path.
bill_path = 'data/schwab_fund.csv'

# Set start dates, existing balance, and contribution interval.
start_date = datetime.date(2025, 1, 1)
balance = 0
interval = 14

Initialize allocator and schedule.

In [4]:
# Initialize the allocator and scheduler.
allocator = SortedAllocator(sort_key='cascade')
scheduler = IndependentScheduler()
lpscheduler = LPScheduler()

Load bills.

In [5]:
# Load the bills as envelopes.
# envelopes = load_envelopes_from_csv(bill_path, interval)
bills = load_bills_from_csv(bill_path)

In [6]:
bills[0].__dict__

{'bill_id': 'suburu_reg',
 'service': 'Car Registraion',
 'amount_due': 191.0,
 'recurring': True,
 'start_date': datetime.date(2025, 5, 12),
 'end_date': None,
 'frequency': 'annual',
 'interval': 1,
 'occurrences': None}

In [7]:
print(bills[0].next_instance(reference_date=datetime.date.today()))

print(bills[0].next_instance(
    reference_date=datetime.date.today()+datetime.timedelta(days=365)
))

print(bills[0].instances_in_range(
    start_reference=datetime.date.today(),
    end_reference=datetime.date.today()+datetime.timedelta(days=365)
))

in_range = bills[0].instances_in_range(
    start_reference=datetime.date.today(),
    end_reference=datetime.date.today()+datetime.timedelta(days=365)
)

print(in_range[0])

print(bills[0].next_instance(reference_date=in_range[0].due_date))

# bills[0]._next_due_date(curr_due_date=datetime.date(2025, 5, 13))

# bills[0]._calculate_occurrences_in_range(
#     start_date=datetime.date(2025, 5, 13),
#     end_date=datetime.date(2026, 5, 31),
#     frequency='annual',
#     interval=1
# )

# bills[0]._increment_date(
#     reference_date=datetime.date(2025, 5, 13),
#     frequency='annual',
#     interval=1,
#     num_intervals=1
# )

# bills[0]._increment_monthly(date=datetime.date(2025, 5, 13), num_months=1) 

BillInstance(bill_id='suburu_reg', service='Car Registraion', amount_due=191.0, due_date=datetime.date(2026, 5, 12))
BillInstance(bill_id='suburu_reg', service='Car Registraion', amount_due=191.0, due_date=datetime.date(2027, 5, 12))
[BillInstance(bill_id='suburu_reg', service='Car Registraion', amount_due=191.0, due_date=datetime.date(2026, 5, 12))]
BillInstance(bill_id='suburu_reg', service='Car Registraion', amount_due=191.0, due_date=datetime.date(2026, 5, 12))
BillInstance(bill_id='suburu_reg', service='Car Registraion', amount_due=191.0, due_date=datetime.date(2027, 5, 12))


In [ ]:
# envelopes[0]#.bill.next_instance()
envelopes[2].next_instance(reference_date=datetime.date(2025, 10, 24))

TypeError: Bill.next_instance() got an unexpected keyword argument 'return_last'

In [ ]:
# envelopes[2].bill._next_due_date(date=datetime.date(2025, 10, 1))
envelopes[2].bill._increment_monthly(date=datetime.date(2025, 10, 24), num_months=6)

datetime.date(2026, 4, 24)

Allocate any existing balance.

In [ ]:
# Allocate existing balance to the envelopes using the cascade sort key.
allocator.allocate(envelopes=envelopes, balance=balance, curr_date=start_date)

for e in envelopes:
    print(
    f"""
    Bill: {e.bill.bill_id}
    Amount Due: {e.bill.amount_due}
    Allocated: {e.allocated}
    Remaining: {e.remaining}
    """
   )


    Bill: suburu_reg
    Amount Due: 191.0
    Allocated: 0
    Remaining: 191.0
    

    Bill: honda_reg
    Amount Due: 334.0
    Allocated: 0
    Remaining: 334.0
    

    Bill: car_insur
    Amount Due: 774.76
    Allocated: 0
    Remaining: 774.76
    

    Bill: prop_tax_1
    Amount Due: 13161.15
    Allocated: 0
    Remaining: 13161.15
    

    Bill: prop_tax_2
    Amount Due: 13161.15
    Allocated: 0
    Remaining: 13161.15
    

    Bill: home_insur
    Amount Due: 3000.0
    Allocated: 0
    Remaining: 3000.0
    


Create contribution schedules.

In [ ]:
# Schedule the contributions.
scheduler.schedule(envelopes=envelopes, start_date=start_date)
# lpscheduler.schedule(envelopes=envelopes, start_date=start_date)

In [ ]:
sum([sum([cf.amount for cf in e.schedule if cf.amount > 0]) for e in envelopes])

30622.059999999998

In [ ]:
for e in envelopes:
    for cf in e.schedule:
        if cf.date == datetime.date(2025, 1, 1):
            print(cf.date, cf.bill_id, cf.amount)

2025-01-01 suburu_reg 20.18
2025-01-01 honda_reg 33.88
2025-01-01 car_insur 95.62
2025-01-01 prop_tax_1 607.05
2025-01-01 prop_tax_2 463.47
2025-01-01 home_insur 96.61


In [ ]:
from collections import defaultdict

date_totals = defaultdict(float)

for e in envelopes:
    for cf in e.schedule:
        if cf.amount > 0:
            date_totals[cf.date] += cf.amount

sum([total for date, total in date_totals.items()])
    # print(date, total)

30622.06